https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html summary:

* `%time`: Time the execution of a single statement
* `%timeit`: Time repeated execution of a single statement for more accuracy
* `%prun`: Run code with the profiler
* `%lprun`: Run code with the line-by-line profiler
* `%memit`: Measure the memory use of a single statement
* `%mprun`: Run code with the line-by-line memory profiler


http://codedelish.com/resources/Python/profiling.html

http://pynash.org/2013/03/06/timing-and-profiling/

In [1]:
import sys
print(sys.version)

3.6.7 |Anaconda, Inc.| (default, Oct 23 2018, 19:16:44) 
[GCC 7.3.0]


In [2]:
!pip install line-profiler
!pip install psutil
!pip install memory_profiler

two functions that we will profile

In [4]:
def mult_by(inpt):
    """
    a simple function that is not complicated
    """
    avalu = inpt*2
    avalu+=1
    return

def primes(n=1000): 
    """
    A more complicated function to be profiled
    """
    A = [True] * (n+1)
    A[0] = False
    A[1] = False
    for i in range(2, int(n**0.5)):
        if A[i]:
            mult_by(i) # not actually relevant to determining a prime number
            for j in range(i**2, n+1, i):
                A[j] = False

    return [x for x in range(2, n) if A[x]]

# time magic function 

https://ipython.readthedocs.io/en/stable/interactive/magics.html

how long does execution of a function take?

`%time` applies to a single line

In [5]:
%time primes(20)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.35 ms


[2, 3, 5, 7, 11, 13, 17, 19]

`%%time` applies to the entire cell

In [6]:
%%time 
primes(20)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 63.4 µs


[2, 3, 5, 7, 11, 13, 17, 19]

Displaying output skews measurement results for cell execution, so store output to a variable

In [7]:
%time reslt=primes(20)

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 30.8 µs


There's not much difference for a small output, but the relevance increases when the input is higher

In [8]:
%time reslt=primes(1000000)

print('length of array:',len(reslt))

CPU times: user 770 ms, sys: 100 ms, total: 870 ms
Wall time: 1.01 s
length of array: 78498


for "microseconds" versus "milliseconds", see https://en.wikipedia.org/wiki/Metric_prefix#List_of_SI_prefixes

To run multiple timing test, use `%timeit`

In [9]:
%timeit reslt=primes(1000000)

479 ms ± 33 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Want to capture the output to a variable? See https://stackoverflow.com/questions/24812253/how-can-i-capture-return-value-with-python-timeit-module

# cProfile

https://docs.python.org/3/library/profile.html

In [6]:
import cProfile

In [7]:
cProfile.run('primes(10000000)')

         451 function calls in 7.122 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      446    0.008    0.000    0.008    0.000 <ipython-input-2-03d799ffa7ac>:1(mult_by)
        1    1.535    1.535    1.535    1.535 <ipython-input-2-03d799ffa7ac>:15(<listcomp>)
        1    5.472    5.472    7.015    7.015 <ipython-input-2-03d799ffa7ac>:5(primes)
        1    0.107    0.107    7.122    7.122 <string>:1(<module>)
        1    0.000    0.000    7.122    7.122 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




# line time

https://nyu-cds.github.io/python-performance-tuning/03-line_profiler/
    
https://jakevdp.github.io/PythonDataScienceHandbook/01.07-timing-and-profiling.html

In [13]:
%load_ext line_profiler

In [14]:
%lprun?

Docstring:
Execute a statement under the line-by-line profiler from the
line_profiler module.

Usage:
  %lprun -f func1 -f func2 <statement>

The given statement (which doesn't require quote marks) is run via the
LineProfiler. Profiling is enabled for the functions specified by the -f
options. The statistics will be shown side-by-side with the code through the
pager once the statement has completed.

Options:

-f <function>: LineProfiler only profiles functions and methods it is told
to profile.  This option tells the profiler about these functions. Multiple
-f options may be used. The argument may be any expression that gives
a Python function or method object. However, one must be careful to avoid
spaces that may confuse the option parser.

-m <module>: Get all the functions/methods in a module

One or more -f or -m options are required to get any useful results.

-D <filename>: dump the raw statistics out to a pickle file on disk. The
usual extension for this is ".lprof". These stat

Profile the lines of the `prime` function

In [15]:
%lprun -f primes primes(10000)

Timer unit: 1e-06 s

Total time: 0.583068 s
File: <ipython-input-8-4330abee72cd>
Function: primes at line 9

Line #      Hits         Time  Per Hit   % Time  Line Contents
     9                                           def primes(n=1000): 
    10                                               """
    11                                               A more complicated function to be profiled
    12                                               """
    13         1        163.0    163.0      0.0      A = [True] * (n+1)
    14         1         15.0     15.0      0.0      A[0] = False
    15         1         14.0     14.0      0.0      A[1] = False
    16        99       1533.0     15.5      0.3      for i in range(2, int(n**0.5)):
    17        98       1514.0     15.4      0.3          if A[i]:
    18        25        519.0     20.8      0.1              mult_by(i)
    19     17006     280086.0     16.5     48.0              for j in range(i**2, n+1, i):
    20     16981     295213.0 

profile use of the `mult_by` function when `primes()` is executed

In [18]:
%lprun -f mult_by primes(10000)

Timer unit: 1e-06 s

Total time: 0.002613 s
File: <ipython-input-8-4330abee72cd>
Function: mult_by at line 1

Line #      Hits         Time  Per Hit   % Time  Line Contents
     1                                           def mult_by(inpt):
     2                                               """
     3                                               a simple function that is not complicated
     4                                               """
     5        25       1266.0     50.6     48.5      avalu = inpt*2
     6        25        893.0     35.7     34.2      avalu+=1
     7        25        454.0     18.2     17.4      return

line_profiler generates profiles based on the lines in a program.

Line based profiling can provide more information about where a program is performing badly.